In [19]:
import joblib
import sys as sys
from sys import argv
import torch
from torch import nn
import torchvision as tv
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import time
import os
import numpy as np
from textdistance import levenshtein as lev
from collections import OrderedDict
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import sampler
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.utils.data import random_split

from PIL import Image

import warnings
warnings.filterwarnings("ignore")


	
path_save_to = '/home/mikhail/Загрузки'
#key_model = ''
name_model = 'resnet'
#path_data = '/content/drive/MyDrive/'
train_path_data = '/home/mikhail/it-academy/Diplom_project/data/train'
test_path_data = '/home/mikhail/it-academy/Diplom_project/data/val'

# Device configuration
device ='cpu'#torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#здесь вычислил std и mean
"""
transform = transforms.Compose([
                                                tv.transforms.Resize((255, 255)),
                                                transforms.ToTensor()
                                            ])

train_data = datasets.ImageFolder(path_data,transform=transform)
train_loader = DataLoader(train_data,batch_size=256)

def get_mean_and_std(loader):
    mean = std = total_images = 0
    for images, _ in loader:
        images = images.view(images.shape[0], images.shape[1], -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += images.shape[0]
    mean /= total_images
    std /= total_images
    return mean, std

#mean, std = get_mean_and_std(train_loader)
#print(mean, std)
"""

#SynthDataset
class SynthDataset(Dataset):
    def __init__(self, path):
        #print(self)
        #print('opt = ',opt)  #выводим сам словарик args
        #super(SynthDataset, self).__init__()   #
        self.path = path #путь к картинкам
        #print(self.path)
        self.images = os.listdir(self.path) #список картинок
        #print(self.images)
        self.nSamples = len(self.images) #число картинок
        #print(self.nSamples)
        f = lambda x: os.path.join(self.path, x)  #объявили лямбда функцию, в этой строке она ничего не делает
        self.imagepaths = list(map(f, self.images)) #список конкатенации пути self.path и картинки 
        #print(self.imagepaths)
        # преобразования изображения
        transform_list =  [transforms.Grayscale(1), # преобразовать изображение в градации серого, 1 - один канал
                            transforms.ToTensor(),  #конвертируем в тензор
                            transforms.Normalize([0.5],[0.5])]  #([0.5652, 0.5957, 0.6045], [0.2579, 0.2609, 0.2609])]  #нормализация тензора
        self.transform = transforms.Compose(transform_list)

    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        assert index <= len(self), 'index range error'
        imagepath = self.imagepaths[index] #путь и картинка
        #print('imagepath = ',imagepath) 
        imagefile = os.path.basename(imagepath) #картинка
        #print('imagefile = ',imagefile)
        img = Image.open(imagepath)
        #print('img = ',img)
        if self.transform is not None:
            img = self.transform(img)
        item = {'img': img, 'idx':index}
        item['label'] = imagefile.split('_')[0]
        return item 

class SynthCollator(object):
    
    def __call__(self, batch):
        #print(batch)
        width = [item['img'].shape[2] for item in batch]
        indexes = [item['idx'] for item in batch]
        imgs = torch.ones([len(batch), batch[0]['img'].shape[0], batch[0]['img'].shape[1], 
                           max(width)], dtype=torch.float32)
        for idx, item in enumerate(batch):
            try:
                imgs[idx, :, :, 0:item['img'].shape[2]] = item['img']
            except:
                print(imgs.shape)
        item = {'img': imgs, 'idx':indexes}
        if 'label' in batch[0].keys():
            labels = [item['label'] for item in batch]
            item['label'] = labels
        return item

train_data = SynthDataset(train_path_data)
#train_data = datasets.ImageFolder(train_path_data,transform=transform)
#train_data_subset = torch.utils.data.Subset(train_data, np.random.choice(len(train_data), 501, replace=False))
train_loader = DataLoader(train_data, batch_size=2, shuffle=True, collate_fn=SynthCollator())
#pbar = tqdm(loader, desc='Epoch: [%d]/[%d] Training'%(self.count, self.epochs), leave=True)
#print(train_loader.dataset)
#for x, y, z in train_loader:
#    #print('items = ', y['img'].shape)
#    print('x = ',type(x))
#    print('y = ',y)
#    print('z = ',z)

test_data = SynthDataset(test_path_data)
#test_data = datasets.ImageFolder(test_path_data,transform=transform)
#test_data_subset = torch.utils.data.Subset(test_data, np.random.choice(len(test_data), 71, replace=False))
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, collate_fn=SynthCollator())


#print(print(train_data_loader.dataset[0][1]))
print('Изображения загрузились в DataLoader')		

Изображения загрузились в DataLoader


In [20]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()
        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)
     
    def forward(self, input):
        #print('BidirectionalLSTM is begin')
        #print('input = ', input)
        #self.rnn.flatten_parameters()
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)
        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)
        #print('BidirectionalLSTM is done')
        return output

class CRNN(nn.Module):

    def __init__(self, num_classes, leakyRelu=False):
        super(CRNN, self).__init__()
        #print(opt)
        #assert opt['imgH'] % 16 == 0, 'imgH has to be a multiple of 16'
        
        ks = [3, 3, 3, 3, 3, 3, 2] #размер ядро свертки, ks=3 означает 3x3 матрица
        ps = [1, 1, 1, 1, 1, 1, 0] #размер padding, padding=1 означает,что размер изображение увеличится на 2
        ss = [1, 1, 1, 1, 1, 1, 1] #stride - шаг, с которым ядро свертки проходит по изображению 
        nm = [64, 128, 256, 256, 512, 512, 512] #количество карт активаций

        cnn = nn.Sequential()

        # 7 layered Convolution network with BatchNorm and ReLU.
        def convRelu(i, batchNormalization=False):
            nIn = 1 if i == 0 else nm[i - 1]  #Количество изображений (карт активаций) на входе
            #print('nIn = ',nIn)
            nOut = nm[i]  #количество карт активаций на выходе
            #print('nOut = ',nOut)
            cnn.add_module('conv{0}'.format(i),
                           nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))  #добавили слой свертки в nn.Sequential() 
            #print(cnn)  #Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
            #padding чтобы дополнить изображение пикселями до исходного размера
            #padding - определяет насколько надо расширить карту признаков перед выполнением свертки. 
            if batchNormalization:
                #print('BatchNorm2d')
                cnn.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))  #Добавили batch нормализацию. Нормализация ускоряет обучение НС,nn.BatchNorm2d 
                #Чтобы привести все значения активации в одну и ту же шкалу, мы нормализуем значения активации таким образом, что скрытое представление не сильно меняется, а также помогает нам улучшить скорость обучения.
                #https://machinelearningmastery.ru/batch-normalization-and-dropout-in-neural-networks-explained-with-pytorch-47d7a8459bcd/?ysclid=lao0b3anjx8079080
            if leakyRelu:
                #print('LeakyReLU')
                cnn.add_module('relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
            else:
                #print('ReLU')
                cnn.add_module('relu{0}'.format(i), nn.ReLU(True)) #функция активации
        #CNN - экстрактор
        convRelu(0) 
        #print('0 = ',cnn)
        cnn.add_module('pooling{0}'.format(0), nn.MaxPool2d(2, 2))  # 64x16x64  kernel_size=2x2 stride=2 default stride=kernel_size
        #Это помогает уменьшить пространственный размер представления, что уменьшает необходимое количество вычислений и весов.
        #print('0 = ',cnn)
        convRelu(1)
        #print('1 = ',cnn)
        cnn.add_module('pooling{0}'.format(1), nn.MaxPool2d(2, 2))  # 128x8x32 kernel_size=2x2 stride=2 default stride=kernel_size
        #print('1 = ',cnn)
        convRelu(2)#, True)
        #print('2 = ',cnn)
        convRelu(3)
        #print('3 = ',cnn)
        cnn.add_module('pooling{0}'.format(2),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 256x4x16 kernel_size=2x2 stride=2x1 - 2 клетки вниз, 1 клетка вправо
        #print('3 = ',cnn)
        convRelu(4, True)
        #print('4 = ',cnn)
        convRelu(5,True)  #было без True
        #print('5 = ',cnn)
        cnn.add_module('pooling{0}'.format(3),
                       nn.MaxPool2d((2, 2), (2, 1), (0, 1)))  # 512x2x16
        #print('5 = ',cnn)
        convRelu(6) #, True)  # 512x1x16
        #print('6 = ',cnn)
        self.cnn = cnn
        #RNN - классификатор
        self.rnn = nn.Sequential()
        self.rnn = nn.Sequential(
            BidirectionalLSTM(512, 256, 256),
            BidirectionalLSTM(256, 256, num_classes))

        #print(self.rnn)
    def forward(self, input):
        # conv features
        #print('train begin')
        #print(input.shape)
        conv = self.cnn(input)
        #print('123')
        b, c, h, w = conv.size()
        #print('b, c, h, w   ',b, c, h, w )
        assert h == 1, "the height of conv must be 1"
        conv = conv.squeeze(2)
        conv = conv.permute(2, 0, 1)  # [w, b, c]
        # rnn features
        output = self.rnn(conv)
        output = output.transpose(1,0) #Tbh to bth
        #print('train done')
        return output


alphabet = '0123456789.-' #классы   #klmnopqrstuvwxyz' #ABCDEFGHIJKLMNOPQRSTUVWXYZ'
num_classes = len(alphabet)  #число классов
num_epochs = 1
learning_rate = 0.001

		
model = CRNN(num_classes = num_classes).to(device)
print('Модель CRNN загружена')

# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #, momentum=0.9) #, weight_decay = 0.001, momentum = 0.9) 
loss = torch.nn.CTCLoss(reduction='mean', zero_infinity=True)
#loss = nn.CrossEntropyLoss(reduction='sum')


#словарик классов, символ:индекс
def alphabet_dict(alphabet):
    alphabet = alphabet + '-'  # for `-1` index

    a_dict = {}
    for i, char in enumerate(alphabet):
        a_dict[char] = i + 1
    #a_dict[''] = 0
    return a_dict

a_dict = alphabet_dict(alphabet)

#закодировать входной набор символов (метка)
def encode(text):
        
        length = []
        result = []
        for item in text:
            # item = item.decode('utf-8', 'strict')
            length.append(len(item))
            for char in item:
                if char in a_dict:
                    index = a_dict[char]
                else:
                    index = 0
                result.append(index)

        text = result
        return (torch.IntTensor(text), torch.IntTensor(length))

# раскодировать предсказанную метку
def decode(t, length, raw=False, alphabet=alphabet):
    #print('alphabet = ',alphabet)
    #print('length.numel() = ',length.numel())
    if length.numel() == 1:
        #print('length = ',length)
        #print('length[0] = ',length[0])   
        length = length[0]

        assert t.numel() == length, "text with length: {} does not match declared length: {}".format(t.numel(),
                                                                                                         length)
        if raw:
            return ''.join([alphabet[i - 1] for i in t])
        else:
            char_list = []
            for i in range(length):
                #print('t[i] = ',t[i])
                #print('t[i - 1] = ',t[i - 1])
                if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                    char_list.append(alphabet[t[i] - 1])
            return ''.join(char_list)
    else:
        # batch mode
        assert t.numel() == length.sum(), "texts with length: {} does not match declared length: {}".format(
            t.numel(), length.sum())
        texts = []
        index = 0
        for i in range(length.numel()):
            #print('length[i] = ',length[i])
            l = length[i]
            #print('t[index:index + l] = ',t[index:index + l])
            #print('torch.IntTensor([l]) = ',torch.IntTensor([l]))
            texts.append(decode(t[index:index + l], torch.IntTensor([l]), raw=raw))
            index += l
        return texts

#точночть посимвольно, т.е. если входное изображение 2222, а предсказано 22, то точность посимвольно 50%
def char_accuracy(pair):
        words, truths = pair
        words, truths = ''.join(words), ''.join(truths)
        sum_edit_dists = lev.distance(words, truths)
        sum_gt_lengths = sum(map(len, truths))
        fraction = 0
        if sum_gt_lengths != 0:
            fraction = sum_edit_dists / sum_gt_lengths

        percent = fraction * 100
        if 100.0 - percent < 0:
            return 0.0
        else:
            return 100.0 - percent

#тест на тестовых значений
def evaluate_accuracy(data_iter, net):
    acc_sum, n = torch.Tensor([0]), 0
    net.eval()
    for X, y in data_iter:
        X = X.to(device)
        y = y.to(device)  
        y_hat = net(X)
        #print(y_hat)
        predicted = y_hat.argmax(axis=1)       
        acc_sum += (predicted== y).sum()
        n += y.shape[0]
    return acc_sum.item() / n


def train(net, train_iter, test_iter, trainer, num_epochs,alphabet):
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        i=1      
        for butch_number, batch in enumerate(train_iter):
            #print('butch_number = ',butch_number)
            #if i%3==0:
            X = batch['img']    # картинка
            y = batch['label']  #метка, т.е цифры на картинке
            X = X.to(device)
            print('y = ',y)
            targets, lengths = encode(y)   #декодируем метку в тензор
            #print(X.shape, y.shape)
            #print('targets = ',targets)    # преобразованная метка
            trainer.zero_grad()
            y_hat = net(X).transpose(1, 0)   #отправляем картинку в модель(свертки+классификатор)
            #print('y_hat = ', y_hat)
            logits = torch.nn.functional.log_softmax(y_hat, 2)  # превращающая логиты в вероятности
            #print('logits = ', logits)
            T, B, H = logits.size()  #17 2 12 ,    17 - количество символов 2 - число картинок, 12 - число классов
            #print('T, B, H = ', T, B, H)
            pred_sizes = torch.LongTensor([T for i in range(B)])  #tensor([17, 17])
            #print('pred_sizes = ', pred_sizes)
            targets = targets.view(-1).contiguous()  #разворачиваем матрицу в вектор
            #print('targets2 = ', targets)
            l = loss(logits, targets, pred_sizes, lengths) #вычисляем loss
            #print('l = ',l.item())
            probs, preds = logits.max(2)  #ищем максимальную вероятность, probs - максимальный элемент в строке матрицы, preds - позиция максимального элемента в строке матрицы
            preds = preds.transpose(1, 0).contiguous().view(-1) # разворачиваем в вектор
            print('preds.data = ',preds.data)
            #print('pred_sizes.data = ',pred_sizes.data)
            #print('alphabet_train = ',alphabet)
            sim_preds = decode(preds.data, pred_sizes.data, raw=False, alphabet=alphabet) #
            print('sim_preds = ',sim_preds)      
            l.backward()
            trainer.step()
            #train_l_sum += l.item()
            #print('123 = ',(y_hat.argmax(axis=1)))
            #print('y = ',y)
            #print('y_hat = ',y_hat)
            #predicted = y_hat.argmax(axis=1) 
            train_acc_sum  = np.mean((list(map(char_accuracy, list(zip(sim_preds, batch['label']))))))
            #print('train_acc_sum = ',train_acc_sum)
            #train_acc_sum += (preds.data == targets).sum().item()
            #n += len(y)
            print("Step. {} Train Loss: {:.3f}. Train acc: {:.3f}".format(i, l,train_acc_sum))   #format(i,
                  #(predicted == y).sum().item() / y.shape[0], l.item(), time.time() -  start))
            i+=1
        print('Эпоха готова') 
        #test_acc = evaluate_accuracy(test_iter, net)

        #print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
        #      'time %.1f sec'
        #      % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
        #         time.time() - start))  
                 
print('обучение:')
train(model, train_loader, test_loader, optimizer, num_epochs,alphabet)  
#print()   
#print('Обучение модели готово.')


Модель CRNN загружена
обучение:
y =  ['-6.0', '-6.5']
preds.data =  tensor([ 6, 11,  8,  8,  8,  8,  8,  8,  8,  8, 11, 11, 11, 11,  6, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 11, 11])
sim_preds =  ['5.7.', '5.']
Step. 1 Train Loss: 6.065. Train acc: 12.500
y =  ['4.7', '-4.4']
preds.data =  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
sim_preds =  ['', '']
Step. 2 Train Loss: 6.225. Train acc: 0.000
y =  ['-1.3', '5.0']
preds.data =  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
sim_preds =  ['', '']
Step. 3 Train Loss: 3.900. Train acc: 0.000
y =  ['5.6', '4.2']
preds.data =  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
sim_preds =  ['', '']
Step. 4 Train Loss: 2.505. Train acc: 0.000
y =  ['3.7', '-2.4']
preds.data =  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])
sim_preds

In [40]:
sim_preds =  ['5151', '5151']
y =  ['3333', '2222']
sim_preds == y

True

In [51]:


sim_preds =  ['5151', '5151']
y =  ['3333', '2222']
list(zip(sim_preds, y))
char_accuracy(('5331', '3333'))

50.0

In [65]:
alphabet_dict(alphabet)

{'0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 '.': 11,
 '-': 13}

In [2]:
!pip3 install lmdb pillow torchvision nltk natsort

Defaulting to user installation because normal site-packages is not writeable


In [16]:
!CUDA_VISIBLE_DEVICES=0 python3 '/home/mikhail/it-academy/github/deep-text-recognition-benchmark/train.py' --train_data '/home/mikhail/it-academy/Diplom_project/train' --valid_data '/home/mikhail/it-academy/Diplom_project/train' --select_data MJ-ST   --batch_ratio 0.5-0.5 --Transformation None --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction CTC --saved_model '/it-academy/Diplom_project/checkpoints/last.pth'

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: /home/mikhail/it-academy/Diplom_project/train
opt.select_data: ['MJ', 'ST']
opt.batch_ratio: ['0.5', '0.5']
--------------------------------------------------------------------------------
dataset_root:    /home/mikhail/it-academy/Diplom_project/train	 dataset: MJ
Traceback (most recent call last):
  File "/home/mikhail/it-academy/github/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/home/mikhail/it-academy/github/deep-text-recognition-benchmark/train.py", line 31, in train
    train_dataset = Batch_Balanced_Dataset(opt)
  File "/home/mikhail/it-academy/github/deep-text-recognition-benchmark/dataset.py", line 42, in __init__
    _dataset, _dataset_log = hierarchical_dataset(root=opt.train_data, opt=opt, select_

In [13]:
!python3 '/home/mikhail/it-academy/github/deep-text-recognition-benchmark/create_lmdb_dataset.py' --inputPath /home/mikhail/it-academy/Diplom_project/data/train --gtFile /home/mikhail/it-academy/Diplom_project/words_numbers.txt --outputPath /home/mikhail/it-academy/Diplom_project/train 

/home/mikhail/it-academy/Diplom_project/data/train/...._22.jpg does not exist
/home/mikhail/it-academy/Diplom_project/data/train/----_23.jpg does not exist
Created dataset with 20 samples


In [17]:
import os
from os import path
import string
name = os.path.basename('/home/mikhail/it-academy/Diplom_project/data/val/-0.0_1.jpg')
name = path.splitext(name)
for oldname in os.listdir('/home/mikhail/it-academy/Diplom_project/data/val/'):
    if oldname.find('-')==0:
        newname = oldname[0:3]+'.'+oldname[3:]
    else:
        newname = oldname[0:2]+'.'+oldname[2:]
    os.rename('/home/mikhail/it-academy/Diplom_project/data/val/'+oldname,'/home/mikhail/it-academy/Diplom_project/data/val/'+newname)

In [14]:
oldname = '-24_49.jpg'
oldname.find('-')

0